In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

sys.path.insert(0, os.path.abspath(os.path.join(os.path.dirname("__file__"), "src")))

In [12]:
from bot.web_interface import WebInterface
from bot.chess_engine import ChessEngine
from bot.chess_bot import ChessBot

In [13]:
# Example setup
stockfish_path = (
    "../stockfish-ubuntu-x86-64-bmi2/stockfish/stockfish-ubuntu-x86-64-bmi2"
)
engine = ChessEngine(stockfish_path)
web_interface = WebInterface()  # assuming you have a browser instance

In [16]:
chess_bot = ChessBot(web_interface, engine)
# Play a game
chess_bot.play_game(playing_as="white", mode="debug", timeout=10, loop_speed = 1)

Best move: e2e4
Performing move: e2 to e4
{'top_left': (268.0, 402.0), 'bottom_right': (335.0, 469.0)}
Dragged from e2 to e4.
Waiting for new computer move... (Current move count: 1)
Waiting for new computer move... (Current move count: 1)
New computer move detected: c6
Detected computer's move: c6
Evaluation after computer's move: {'type': 'cp', 'value': 28}
Best move: d2d4
Performing move: d2 to d4
{'top_left': (201.0, 402.0), 'bottom_right': (268.0, 469.0)}
Dragged from d2 to d4.
Waiting for new computer move... (Current move count: 2)
Waiting for new computer move... (Current move count: 3)
New computer move detected: d5
Detected computer's move: d5
Evaluation after computer's move: {'type': 'cp', 'value': 44}
Best move: e4e5
Performing move: e4 to e5
{'top_left': (268.0, 268.0), 'bottom_right': (335.0, 335.0)}
Dragged from e4 to e5.
Waiting for new computer move... (Current move count: 4)
Waiting for new computer move... (Current move count: 5)
New computer move detected: Bf5
Dete

TypeError: expected string or bytes-like object

In [ ]:
uddir = "/Users/mdicio/Library/Application Support/Google/Chrome"
profile = "Profile 1"

if os.path.exists(f"{uddir}/{profile}"):
    print(1)

u_options = uc.ChromeOptions()
u_options.add_argument(f"user-data-dir={uddir}")
u_options.add_argument(f"--profile-directory={profile}")

browser = uc.Chrome(
    browser_executable_path="/usr/bin/google-chrome-beta",
    options=u_options,
    use_subprocess=True,
)
time.sleep(2.8)
HOME_URL = "https://lichess.org/"
browser.get(HOME_URL)

In [ ]:
# navigate_to_computer_play(browser)
# select_computer_level(browser, level=8)
# select_color(browser, color="white")

In [27]:
play_game(engine, playing_as="white")

Best move: e2e4
Performing move: e2 to e4
{'top_left': (268.0, 402.0), 'bottom_right': (335.0, 469.0)}
Dragged from e2 to e4.
move count according to table 1
move count according to game logic 1
Waiting for new computer move... (Current move count: 1)
Waiting for new computer move... (Current move count: 1)
Waiting for new computer move... (Current move count: 1)
Waiting for new computer move... (Current move count: 1)
Waiting for new computer move... (Current move count: 1)
Waiting for new computer move... (Current move count: 1)
Waiting for new computer move... (Current move count: 1)
Waiting for new computer move... (Current move count: 1)
Waiting for new computer move... (Current move count: 1)
Waiting for new computer move... (Current move count: 1)
Waiting for new computer move... (Current move count: 1)
Waiting for new computer move... (Current move count: 1)
Waiting for new computer move... (Current move count: 1)
Waiting for new computer move... (Current move count: 1)
Waiting

KeyboardInterrupt: 